In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import convert_all as crd

In [3]:
# get all low_dim data
task_name = "mujoco_stack_cups"
raw_root_dir = "../data/raw"
raw_dir = f"{raw_root_dir}/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["obs_action.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    name_converter={
        "/obs/jq": "/observations/qpos",
        "/act": "/action",
    },
    pre_process=None,
    concatenater=None,
    key_filter=[
        "/time",
        "/obj_pose/cup_blue",
        "/obj_pose/cup_pink",
        "/obj_pose/arm_base",
        *[f"/obj_pose/link{i}" for i in range(1, 7)],
        "/obj_pose/right",
        "/obj_pose/left",
        "/obj_pose/camera"
    ],
)

Data Converting: 100%|██████████| 50/50 [00:00<00:00, 114.00it/s]


In [4]:
print(len(list(data.keys())))
print(data.keys())
print(data['000'].keys())
print(data['000']['/observations/qpos'][50])
print(len(data['000']['/observations/qpos']))

50
dict_keys(['007', '018', '047', '010', '046', '029', '026', '032', '030', '035', '014', '037', '015', '011', '016', '022', '039', '023', '048', '012', '028', '000', '044', '036', '002', '049', '008', '001', '024', '038', '043', '004', '009', '025', '021', '034', '041', '027', '045', '006', '013', '031', '020', '042', '017', '033', '019', '040', '005', '003'])
dict_keys(['/time', '/observations/qpos', '/action'])
[0.4991617040200507, -1.335123790302071, 0.6402408818255219, -1.5707059366500984, 0.9332356888993104, 2.0700801708531023, 0.9312934013046861]
294


In [5]:
import os

# merge high_dim data and save
raw_dir
names = ["video_0", "video"]
video_names = [f"{names[0]}.mp4", f"{names[1]}.mp4"]
target_root_dir = "../data/raw"
target_dir = f"{target_root_dir}/{task_name}_raw"
low_dim_data = data
name_converter = {
    names[0]: "/observations/images/0",
    names[1]: "/observations/images/1",
}
target_namer = lambda i: f"{i}"

compresser = None

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))

max_pad_lenth = max(episode_lens)

# save all data
episode_names = list(low_dim_data.keys())
print(f"Episode lengths: {episode_lens}")
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")


def save_one(index, ep_name):
    crd.merge_video_and_save(
        low_dim_data[ep_name],
        f"{raw_dir}/{ep_name}",
        video_names,
        crd.save_dict_to_json_and_mp4,
        name_converter,
        compresser,
        f"{target_dir}/" + target_namer(index),
        max_pad_lenth,
        fps=15,
    )
    data.pop(ep_name, None)

Episode lengths: [283, 286, 289, 288, 285, 281, 284, 285, 280, 284, 286, 288, 290, 290, 289, 290, 283, 286, 284, 285, 285, 294, 288, 288, 288, 291, 287, 289, 286, 287, 290, 284, 288, 284, 287, 283, 287, 285, 287, 291, 292, 284, 286, 286, 282, 289, 285, 286, 283, 283]
Max episode length: 294
All episodes: ['007', '018', '047', '010', '046', '029', '026', '032', '030', '035', '014', '037', '015', '011', '016', '022', '039', '023', '048', '012', '028', '000', '044', '036', '002', '049', '008', '001', '024', '038', '043', '004', '009', '025', '021', '034', '041', '027', '045', '006', '013', '031', '020', '042', '017', '033', '019', '040', '005', '003']
episode number: 50


In [6]:
# save all
from concurrent.futures import ThreadPoolExecutor, as_completed
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:
    for index, ep_name in enumerate(episode_names):  # [:2]
        #Note: exception will not be printed
        futures.append(executor.submit(save_one, index, ep_name))
# Wait for all futures to complete and print any exceptions
for future in as_completed(futures):
    future.result()
print(f"All data saved to {target_dir}")

Call: save_dict_to_json_and_mp4.
target_path= ../data/raw/mujoco_stack_cups_raw/1
Save images to ../data/raw/mujoco_stack_cups_raw/1/0.avi with w_h=(640, 480), fps=15
Call: save_dict_to_json_and_mp4.
target_path= ../data/raw/mujoco_stack_cups_raw/3
Save images to ../data/raw/mujoco_stack_cups_raw/3/0.avi with w_h=(640, 480), fps=15
Call: save_dict_to_json_and_mp4.
target_path= ../data/raw/mujoco_stack_cups_raw/10
Save images to ../data/raw/mujoco_stack_cups_raw/10/0.avi with w_h=(640, 480), fps=15
Call: save_dict_to_json_and_mp4.
target_path= ../data/raw/mujoco_stack_cups_raw/13
Save images to ../data/raw/mujoco_stack_cups_raw/13/0.avi with w_h=(640, 480), fps=15
Save the video data to image_path.
Save images to ../data/raw/mujoco_stack_cups_raw/1/1.avi with w_h=(640, 480), fps=15
Call: save_dict_to_json_and_mp4.
target_path= ../data/raw/mujoco_stack_cups_raw/20
Save images to ../data/raw/mujoco_stack_cups_raw/20/0.avi with w_h=(640, 480), fps=15
Call: save_dict_to_json_and_mp4.
target

In [ ]:
save_one(0, episode_names[0])

In [ ]:
# check hdf5 data
import convert_all as crd
task_name = "act_airbot_play_gs"
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
data = crd.hdf5_to_dict(f"{target_dir}/episode_0.hdf5")
print(data.keys())
data_flat = crd.flatten_dict(data, prefix="/")
print(data_flat.keys())

In [ ]:
print(data_flat['/compressed_len'])

In [ ]:
# show actions
start = 120
end = start + 3

action = data_flat["/action"][start:end]
obs = data_flat["/observations/qpos"][start+1:end+1]
print(action)
print(obs)
print((obs - action))

In [ ]:
print(action[1])
print(obs[1])
print((obs[1] - action[1]) * 180 / 3.1415926)

In [ ]:
import cv2
image = data_flat["/observations/images/0"][0]
print(image.shape)
compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)
image_dc = compresser.decompress(image, "jpg")
print(image_dc.shape)
import matplotlib.pyplot as plt
plt.imshow(image_dc[:, :, ::-1])
plt.show()